In [38]:
just_checking_integrity=False

In [39]:
import gc
import pandas as pd
import tensorflow_ranking as tfr
import scipy.sparse as sps

In [40]:
valid=sps.load_npz("../dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [41]:
# Create different datasets based on the selected list in order to compare between them
selected_list = ["UCF_W","ICBF_W","Graph_W","ICF_W"]

for model in selected_list:
    globals()[f"dataset_{model}"]=None
    globals()[f"dataset_max_{model}"]=None
#["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]

for model in selected_list:
    df_max = None
    df_feats = None
    print(gc.collect())
    print(f"loading {model}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/train/{model}.csv")
    print(len(df))
    print(f"loaded {model}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]

        if df_feats is not None:
            globals()[f"dataset_{model}"]=df_feats
        if df_max is not None:
            globals()[f"dataset_max_{model}"]=df_max
    del df
        

#del dataset_max
del df_feats
del df_max

581
loading UCF_W
18588456
loaded UCF_W
0
loading ICBF_W
16931962
loaded ICBF_W
0
loading Graph_W
9381571
loaded Graph_W
0
loading ICF_W
20341641
loaded ICF_W


In [42]:
dataset_UCF_W.head()

,Session_Id,Item_ID,Score_UCF_W
0,918382,78,0.515642
1,918382,178,0.281229
2,918382,230,0.127506
3,918382,242,0.395681
4,918382,292,0.200113


In [43]:
dataset_max_UCF_W.head()

,Session_Id,Max_Score_UCF_W
0,918382,60.744503
209,918383,594.028260
439,918384,70.502235
662,918385,534.638400
921,918386,482.758480


In [44]:
dataset_ICF_W.head()

,Session_Id,Item_ID,Score_ICF_W
0,918382,44,0.019639
1,918382,78,0.587459
2,918382,103,0.027119
3,918382,143,0.003369
4,918382,145,0.020562


In [45]:
dataset_max_ICF_W.head()

,Session_Id,Max_Score_ICF_W
0,918382,0.395666
278,918383,1.092114
494,918384,0.294585
722,918385,1.956541
1005,918386,0.728324


In [46]:
dataset_ICBF_W.head()

,Session_Id,Item_ID,Score_ICBF
0,918382,44,0.007850
1,918382,46,0.639459
2,918382,80,0.075177
3,918382,81,0.792410
4,918382,96,0.059291


In [47]:
dataset_max_ICBF_W.head()

,Session_Id,Max_Score_ICBF
0,918382,3.067216
226,918383,7.204127
401,918384,3.193477
604,918385,17.122883
877,918386,6.329576


In [48]:
dataset_Graph_W.head()

,Session_Id,Item_ID,Score_Graph_W
0,918382,78,0.377850
1,918382,178,0.142546
2,918382,230,0.055109
3,918382,242,0.273503
4,918382,292,0.105606


In [49]:
dataset_max_Graph_W.head()

,Session_Id,Max_Score_Graph_W
0,918382,1.982604
107,918383,91.798250
211,918384,4.312901
318,918385,142.414630
428,918386,49.553764


In [50]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/train/candidates.csv")
if just_checking_integrity:
    submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
else:
    submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [51]:
submission_df_tr.head()

,Session_Id,Item_ID,score_transformer,rank_transformer
0,918382,2785,0.061407,1
0,918382,3731,0.039482,2
0,918382,3355,0.023729,3
0,918382,1458,0.023356,4
0,918382,3820,0.019259,5


In [52]:
for model in selected_list:
    globals()[f"dataset_{model}"]=globals()[f"dataset_{model}"].merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
    globals()[f"dataset_{model}"]=globals()[f"dataset_{model}"].merge(globals()[f"dataset_max_{model}"],on=["Session_Id"],how="left")

In [53]:
if just_checking_integrity:
    for model in selected_list:
        globals()[f"dataset_{model}"]=globals()[f"dataset_{model}"][:int(0.5*len(globals()[f"dataset_{model}"]))]

In [54]:
for model in selected_list:
    del globals()[f"dataset_max_{model}"]
#del submission_df
del submission_df_tr

In [55]:
import numpy as np

for model in selected_list:
    globals()[f"sessions_{model}"]=np.unique(globals()[f"dataset_{model}"]["Session_Id"])
    globals()[f"train_{model}"]=pd.DataFrame()
    globals()[f"train_{model}"]["Session_Id"]=globals()[f"sessions_{model}"]
    if just_checking_integrity:
        globals()[f"train_{model}"]["Item_ID"]=bought[:len(globals()[f"sessions_{model}"])]
    else:
        globals()[f"train_{model}"]["Item_ID"]=bought
    globals()[f"train_{model}"]["target"]=1

In [56]:
gc.collect()

3509

In [57]:
for model in selected_list:
    globals()[f"dataset_{model}"]=globals()[f"dataset_{model}"].merge(globals()[f"train_{model}"],on=["Session_Id","Item_ID"],how="left")
    globals()[f"dataset_{model}"].fillna(0,inplace=True)
    gc.collect()

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_30784/57251536.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  globals()[f"dataset_{model}"].fillna(0,inplace=True)
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_30784/57251536.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  globals()[f"dataset_{model}"].fillna(0,inplace=True)
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_30784/57251536.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future versio

In [58]:
for model in selected_list:
    globals()[f"dataset_{model}"]["target"]=globals()[f"dataset_{model}"]["target"].astype("int8")
    gc.collect()

In [59]:
for model in selected_list:
    globals()[f"sessions_containing_bought_among_candidates_{model}"]=globals()[f"dataset_{model}"][["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
    globals()[f"sessions_containing_bought_among_candidates_{model}"]=globals()[f"sessions_containing_bought_among_candidates_{model}"].loc[globals()[f"sessions_containing_bought_among_candidates_{model}"]["target"]>0]
    
    globals()[f"dataset_{model}"]=globals()[f"dataset_{model}"][globals()[f"dataset_{model}"]['Session_Id'].isin(globals()[f"sessions_containing_bought_among_candidates_{model}"]["Session_Id"].tolist())]

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_30784/2495118367.py:2: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  globals()[f"sessions_containing_bought_among_candidates_{model}"]=globals()[f"dataset_{model}"][["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_30784/2495118367.py:2: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  globals()[f"sessions_containing_bought_among_candidates_{model}"]=globals()[f"dataset_{model}"][["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
/var/folders/yt/h_zrk_k931s4wj_9

In [60]:
NHITS_UCF_W=len(dataset_UCF_W['Session_Id'].unique())
HITRATE_UCF_W=NHITS_UCF_W/81600 #approximate value 
HITRATE_UCF_W

0.5932107843137255

In [61]:
NHITS_ICBF_W=len(dataset_ICBF_W['Session_Id'].unique())
HITRATE_ICBF_W=NHITS_ICBF_W/81600 #approximate value 
HITRATE_ICBF_W

0.6461642156862745

In [62]:
NHITS_ICF_W=len(dataset_ICF_W['Session_Id'].unique())
HITRATE_ICF_W=NHITS_ICF_W/81600 #approximate value 
HITRATE_ICF_W

0.6713112745098039

In [63]:
NHITS_Graph_W=len(dataset_Graph_W['Session_Id'].unique())
HITRATE_Graph_W=NHITS_Graph_W/81600 #approximate value 
HITRATE_Graph_W

0.614656862745098

In [64]:
dataset_UCF_W.head(20)

,Session_Id,Item_ID,Score_UCF_W,score_transformer,rank_transformer,Max_Score_UCF_W,target
269,918383,13,0.000000,0.001024,88.0,594.02826,0
270,918383,55,0.054177,0.000000,0.0,594.02826,0
271,918383,122,0.025767,0.000000,0.0,594.02826,0
272,918383,189,0.129413,0.000000,0.0,594.02826,0
273,918383,215,0.036236,0.000000,0.0,594.02826,0
274,918383,220,0.000000,0.001056,86.0,594.02826,0
275,918383,232,0.000000,0.049706,4.0,594.02826,0
276,918383,260,0.039900,0.000000,0.0,594.02826,0
277,918383,320,0.040230,0.005522,24.0,594.02826,0
278,918383,354,0.000000,0.001002,93.0,594.02826,0


In [65]:
dataset_UCF_W

,Session_Id,Item_ID,Score_UCF_W,score_transformer,rank_transformer,Max_Score_UCF_W,target
269,918383,13,0.000000,0.001024,88.0,594.028260,0
270,918383,55,0.054177,0.000000,0.0,594.028260,0
271,918383,122,0.025767,0.000000,0.0,594.028260,0
272,918383,189,0.129413,0.000000,0.0,594.028260,0
273,918383,215,0.036236,0.000000,0.0,594.028260,0
...,...,...,...,...,...,...,...
23199176,999998,13380,0.251168,0.001240,100.0,81.306076,0
23199177,999998,13590,0.000000,0.000000,0.0,81.306076,0
23199178,999998,14643,0.000000,0.000000,0.0,81.306076,0
23199179,999998,14659,0.000000,0.002818,39.0,81.306076,0


In [66]:
dataset_ICBF_W.head(20)

,Session_Id,Item_ID,Score_ICBF,score_transformer,rank_transformer,Max_Score_ICBF,target
303,918383,13,0.000000,0.001024,88.0,7.204127,0
304,918383,34,0.181000,0.000000,0.0,7.204127,0
305,918383,52,0.015594,0.000000,0.0,7.204127,0
306,918383,55,0.632362,0.000000,0.0,7.204127,0
307,918383,65,0.274329,0.000000,0.0,7.204127,0
308,918383,112,0.045952,0.000000,0.0,7.204127,0
309,918383,169,0.839069,0.000000,0.0,7.204127,0
310,918383,220,0.000000,0.001056,86.0,7.204127,0
311,918383,232,0.000000,0.049706,4.0,7.204127,0
312,918383,251,0.319064,0.000000,0.0,7.204127,0


In [67]:
dataset_ICBF_W

,Session_Id,Item_ID,Score_ICBF,score_transformer,rank_transformer,Max_Score_ICBF,target
303,918383,13,0.000000,0.001024,88.0,7.204127,0
304,918383,34,0.181000,0.000000,0.0,7.204127,0
305,918383,52,0.015594,0.000000,0.0,7.204127,0
306,918383,55,0.632362,0.000000,0.0,7.204127,0
307,918383,65,0.274329,0.000000,0.0,7.204127,0
...,...,...,...,...,...,...,...
22804798,999999,7028,0.280848,0.000000,0.0,9.421845,0
22804799,999999,7208,0.000000,0.001670,75.0,9.421845,0
22804800,999999,11657,0.000000,0.002732,45.0,9.421845,0
22804801,999999,12609,0.844470,0.000000,0.0,9.421845,0


In [68]:
dataset_ICF_W.head(20)

,Session_Id,Item_ID,Score_ICF_W,score_transformer,rank_transformer,Max_Score_ICF_W,target
332,918383,13,0.000000,0.001024,88.0,1.092114,0
333,918383,55,0.018442,0.000000,0.0,1.092114,0
334,918383,107,0.005707,0.000000,0.0,1.092114,0
335,918383,189,0.278846,0.000000,0.0,1.092114,0
336,918383,220,0.000000,0.001056,86.0,1.092114,0
337,918383,232,0.000000,0.049706,4.0,1.092114,0
338,918383,253,0.000005,0.000000,0.0,1.092114,0
339,918383,260,0.012055,0.000000,0.0,1.092114,0
340,918383,320,0.000000,0.005522,24.0,1.092114,0
341,918383,354,0.000000,0.001002,93.0,1.092114,0


In [69]:
dataset_ICF_W

,Session_Id,Item_ID,Score_ICF_W,score_transformer,rank_transformer,Max_Score_ICF_W,target
332,918383,13,0.000000,0.001024,88.0,1.092114,0
333,918383,55,0.018442,0.000000,0.0,1.092114,0
334,918383,107,0.005707,0.000000,0.0,1.092114,0
335,918383,189,0.278846,0.000000,0.0,1.092114,0
336,918383,220,0.000000,0.001056,86.0,1.092114,0
...,...,...,...,...,...,...,...
24993609,999999,15625,0.138915,0.000000,0.0,0.786831,0
24993610,999999,16357,0.005289,0.000000,0.0,0.786831,0
24993611,999999,16479,0.541090,0.000000,0.0,0.786831,0
24993612,999999,16520,0.359696,0.000000,0.0,0.786831,0


In [70]:
dataset_Graph_W.head(20)

,Session_Id,Item_ID,Score_Graph_W,score_transformer,rank_transformer,Max_Score_Graph_W,target
170,918383,13,0.000000,0.001024,88.0,91.79825,0
171,918383,55,0.005355,0.000000,0.0,91.79825,0
172,918383,189,0.020602,0.000000,0.0,91.79825,0
173,918383,220,0.000000,0.001056,86.0,91.79825,0
174,918383,232,0.000000,0.049706,4.0,91.79825,0
175,918383,260,0.003595,0.000000,0.0,91.79825,0
176,918383,320,0.000000,0.005522,24.0,91.79825,0
177,918383,354,0.000000,0.001002,93.0,91.79825,0
178,918383,356,0.000000,0.005647,23.0,91.79825,0
179,918383,357,0.003286,0.000000,0.0,91.79825,0


In [71]:
dataset_Graph_W

,Session_Id,Item_ID,Score_Graph_W,score_transformer,rank_transformer,Max_Score_Graph_W,target
170,918383,13,0.000000,0.001024,88.0,91.798250,0
171,918383,55,0.005355,0.000000,0.0,91.798250,0
172,918383,189,0.020602,0.000000,0.0,91.798250,0
173,918383,220,0.000000,0.001056,86.0,91.798250,0
174,918383,232,0.000000,0.049706,4.0,91.798250,0
...,...,...,...,...,...,...,...
14393555,999999,6101,0.010951,0.000000,0.0,12.826448,0
14393556,999999,7208,0.000000,0.001670,75.0,12.826448,0
14393557,999999,11657,0.000000,0.002732,45.0,12.826448,0
14393558,999999,12609,0.653486,0.000000,0.0,12.826448,0


In [72]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)

for model in selected_list:
    globals()[f"dataset_{model}"] = globals()[f"dataset_{model}"].merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [73]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [75]:
from sklearn.model_selection import KFold
import lightgbm
import os
k=10

score = {}

for model_name in selected_list:
    score[model_name]=0
    kf = KFold(n_splits=k)
    index=0
    print(f"Running now on model {model_name}")
    for train_index, valid_index in kf.split(globals()[f"train_{model_name}"]):
    
        train_sessions=globals()[f"train_{model_name}"]["Session_Id"][train_index].tolist()
        valid_sessions=globals()[f"train_{model_name}"]["Session_Id"][valid_index].tolist()
    
        train_set=globals()[f"dataset_{model_name}"][globals()[f"dataset_{model_name}"]["Session_Id"].isin(train_sessions)]
        valid_set=globals()[f"dataset_{model_name}"][globals()[f"dataset_{model_name}"]["Session_Id"].isin(valid_sessions)]
    
        train_set=train_set.sort_values("Session_Id")
        valid_set=valid_set.sort_values("Session_Id")
        
        qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
        qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
        
        X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
        y_train=train_set["target"]
        
        X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
        y_valid=valid_set["target"]
    
        model = lightgbm.LGBMRanker(
        objective="lambdarank",
        #objective="rank_xendcg",
        #metric="mrr",
        n_estimators=2000,
        lambdarank_truncation_level=103,
        reg_alpha=1e-4,
        learning_rate=0.05,
        min_child_samples=400,
        subsample=0.8,
        subsample_freq=20
        )
        ranker=model.fit(
            X=X_train,
            y=y_train,
            group=qids_train,
            eval_set=[(X_valid, y_valid)],
            eval_group=[qids_valid],
            eval_at=(100),
            callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
            #verbose=10,
            eval_metric=["map","ndcg"]
        )
        ranker.booster_.save_model(f'../rankers/model_{model_name}_{index}.txt')
        #Get maximum val score
        print("=================")
        print(ranker._evals_result["valid_0"])
        score[model_name]+=np.max(ranker._evals_result["valid_0"]["map@100"])
        index+=1

Running now on model UCF_W
[LightGBM] [Info] Total groups: 43071, total data: 12210450
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130230 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2797
[LightGBM] [Info] Number of data points in the train set: 12210450, number of used features: 13
[LightGBM] [Info] Total groups: 5335, total data: 1519019
[1]	valid_0's map@100: 0.193881	valid_0's ndcg@100: 0.324202
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.251944	valid_0's ndcg@100: 0.370931
[3]	valid_0's map@100: 0.257768	valid_0's ndcg@100: 0.376438
[4]	valid_0's map@100: 0.258483	valid_0's ndcg@100: 0.377218
[5]	valid_0's map@100: 0.257993	valid_0's ndcg@100: 0.37714
[6]	valid_0's map@100: 0.259098	valid_0's ndcg@100: 0.377825
[7]	valid_0's map@100: 0.259226	valid_0's ndcg@100: 0.377954
[8]	valid_0'

In [76]:
final_performcance_UCF_W=score["UCF_W"]/k
print("overall MAP@100 UCF_W = ", final_performcance_UCF_W)

overall MAP@100 UCF_W =  0.2859742326417465


In [77]:
print("normalized score UCF_W ", final_performcance_UCF_W*HITRATE_UCF_W)

normalized score UCF_W  0.16964299883892625


In [78]:
final_performcance_ICF_W=score["ICF_W"]/k
print("overall MAP@100 ICF_W = ", final_performcance_ICF_W)

overall MAP@100 ICF_W =  0.2564834684516078


In [79]:
print("normalized score ICF_W ", final_performcance_ICF_W*HITRATE_ICF_W)

normalized score ICF_W  0.1721802440969439


In [80]:
final_performcance_ICBF_W=score["ICBF_W"]/k
print("overall MAP@100 ICBF_W = ", final_performcance_ICBF_W)

overall MAP@100 ICBF_W =  0.25536156677643684


In [81]:
print("normalized score ICBF_W ", final_performcance_ICBF_W*HITRATE_ICBF_W)

normalized score ICBF_W  0.16500550651251453


In [82]:
final_performcance_Graph_W=score["Graph_W"]/k
print("overall MAP@100 Graph_W = ", final_performcance_Graph_W)

overall MAP@100 Graph_W =  0.28654595349164746


In [84]:
print("normalized score Graph_W ", final_performcance_Graph_W*HITRATE_Graph_W)

normalized score Graph_W  0.1761274368054788


In [93]:
compression_table = pd.DataFrame(
    [
        [
              final_performcance_UCF_W,
              final_performcance_ICF_W,
              final_performcance_ICBF_W,
              final_performcance_Graph_W
        ],
        [
            final_performcance_UCF_W*HITRATE_UCF_W,
            final_performcance_ICF_W*HITRATE_ICF_W,
            final_performcance_ICBF_W*HITRATE_ICBF_W,
            final_performcance_Graph_W*HITRATE_Graph_W
        ]
    ],
    index=['MAP@100', "Normalized Score"],
    columns=['UCF_W+Transformer', 'ICF_W+Transformer', 'ICBF_W+Transformer', 'Graph_W+Transformer'])

In [94]:
compression_table

,UCF_W+Transformer,ICF_W+Transformer,ICBF_W+Transformer,Graph_W+Transformer
MAP@100,0.285974,0.256483,0.255362,0.286546
Normalized Score,0.169643,0.172180,0.165006,0.176127
